In [1]:
# TODO 
from robotodo.engines.isaac.scene import Scene

scene = Scene.load(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/default_environment.usd",
)
# scene.autostepping = True

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'

|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------

2025-11-28T03:03:14Z [13,716ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed
2025-11-28T03:03:15Z [14,955ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-11-28T03:03:15Z [14,955ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-11-28T03:03:15Z [15,073ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


In [2]:
from robotodo.engines.isaac.articulation import Articulation

panda = Articulation.load(
    "/TODOPanda{0..1000}",
    "/home/sysadmin/lab/robotodo/__todo__/todo-202510xx-misc/todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf",
    scene=scene,
)

In [8]:
panda.dof_positions = 1
panda.driver.dof_target_positions = 1

2025-11-28T03:06:04Z [184,009ms] [Error] [omni.physx.plugin] PhysX error: The application needs to increase PxGpuDynamicsMemoryConfig::foundLostPairsCapacity to 501501, otherwise, the simulation will miss interactions
, FILE /builds/omniverse/physics/physx/source/gpubroadphase/src/PxgCudaBroadPhaseSap.cpp, LINE 497
2025-11-28T03:06:04Z [184,009ms] [Error] [omni.physx.plugin] PhysX error: The application needs to increase PxGpuDynamicsMemoryConfig::foundLostAggregatePairsCapacity to 262144, otherwise, the simulation will miss interactions, FILE /builds/omniverse/physics/physx/source/gpubroadphase/src/PxgAABBManager.cpp, LINE 1273
2025-11-28T03:06:20Z [199,798ms] [Error] [omni.physx.plugin] PhysX error: The application needs to increase PxGpuDynamicsMemoryConfig::foundLostAggregatePairsCapacity buffers to 3599, otherwise, the simulation will miss interactions, FILE /builds/omniverse/physics/physx/source/gpubroadphase/src/PxgAABBManager.cpp, LINE 1284
2025-11-28T03:06:20Z [199,902ms] [Err

In [11]:
%%timeit -n 1

panda.dof_positions 

The slowest run took 4.08 times longer than the fastest. This could mean that an intermediate result is being cached.
319 μs ± 204 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
planner = panda.planner(
    base_link="panda_link0",
    end_link="panda_hand",

    # base_link="base_link",
    # end_link="ee_link",

    # use_self_collision=True,
    # use_world_collision=True,
    
    # _curobo_motiongen_config=dict(
    #     # self_collision_check=False
    # ),
)

In [19]:
from robotodo.utils.pose import Pose

await panda.driver.execute_action(
    planner.compute_action({"target_pose": Pose([.5, .2, .5])}),
    position_error_limit=1,
    velocity_error_limit=1,
)

Batch mode enable graph is only supported with num_graph_seeds==1


In [ ]:
from robotodo.engines.isaac._kernel import Kernel
from robotodo.engines.isaac._utils.usd import usd_add_reference

import os
import tempfile


def usd_import_urdf(
    stage: "pxr.Usd.Stage",
    path: str,
    resource_or_model: str,
    kernel: Kernel,
):
    pxr = kernel.pxr
    isaacsim = kernel.isaacsim
    # TODO rm
    import isaacsim.asset.importer.urdf

    with tempfile.TemporaryDirectory() as tmpdir:
        usd_path = os.path.join(tmpdir, "todo.usd")

        urdf_import_config = isaacsim.asset.importer.urdf.URDFCreateImportConfig().do()
        urdf_import_config.make_default_prim = True  # Make the robot the default prim in the scene
        # import_config.fix_base = config.get("fix_root_link", False) # Fix the base of the robot to the ground
        urdf_import_config.merge_fixed_joints = False
        # import_config.convex_decomp = False  # Disable convex decomposition for simplicity
        # import_config.self_collision = False  # Disable self-collision for performance

        isaacsim.asset.importer.urdf.URDFParseAndImportFile(
            urdf_path=resource_or_model,
            # TODO
            import_config=urdf_import_config,
            dest_path=usd_path,
        ).do()

        # TODO NOTE this alt impl does not respect merge_fixed_joints due to a bug?
        # urdf_robot_model = isaacsim.asset.importer.urdf.URDFParseFile(
        #     resource_or_model,
        #     import_config=urdf_import_config,
        # ).do()
        # isaacsim.asset.importer.urdf.URDFImportRobot(
        #     urdf_robot=urdf_robot_model,
        #     dest_path=usd_path,
        # ).do()

        layer = pxr.Usd.Stage.Open(usd_path).Flatten()

        return usd_add_reference(
            stage,
            path=path,
            # resource=usd_path,
            resource=layer.identifier,
            kernel=kernel,
        )
    


In [24]:
usd_import_urdf(
    scene._usd_stage,
    "/todo2",
    "/home/sysadmin/lab/robotodo/__todo__/todo-202510xx-misc/todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf",
    kernel=scene._kernel,
)

Usd.Prim(</todo2>)

In [36]:
scene.load("/tmp/tmpaf2fccao/todo.usd")

Scene()

2025-11-27T11:28:51Z [1,837,081ms] [Error] [omni.physicsschema.plugin] Joint (/panda/joints/panda_finger_joint1) body relationship /panda/panda_hand points to a non existent prim, joint will not be created.
2025-11-27T11:28:51Z [1,837,081ms] [Error] [omni.physicsschema.plugin] Joint (/panda/joints/panda_finger_joint2) body relationship /panda/panda_hand points to a non existent prim, joint will not be created.
2025-11-27T11:28:51Z [1,837,081ms] [Error] [omni.physicsschema.plugin] Joint (/panda/joints/panda_joint1) body relationship /panda/panda_link0 points to a non existent prim, joint will not be created.


In [ ]:
import weakref

import os
import tempfile


_todo_weakrefs = weakref.WeakValueDictionary()

def todo(stage):
    tmpdir = tempfile.TemporaryDirectory()
    usd_path = os.path.join(tmpdir.name, "todo.usd")
    # _todo_weakrefs[tmpdir] = stage
    # stage._todo = tmpdir
    return tmpdir.name

todo(scene._usd_stage)

'/tmp/tmpux9n13t8'

In [2]:
import os
import tempfile

tmpdir = tempfile.TemporaryDirectory()
# usd_path = os.path.join(tmpdir, "todo.usd")

tmpdir

<TemporaryDirectory '/tmp/tmp63voxr_k'>

In [11]:
import isaacsim.asset.importer.urdf

urdf_robot_model = isaacsim.asset.importer.urdf.URDFParseFile(
    "/home/sysadmin/lab/robotodo/__todo__/todo-202510xx-misc/todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf"
).do()

isaacsim.asset.importer.urdf.URDFImportRobot(
    urdf_robot=urdf_robot_model,
    dest_path="/tmp/todo-robot/todo.usd",
).do()

'/panda'